In [3]:
conf_dw = {
      'url':'jdbc:mysql://localhost:3306/final_project?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigMysql',
      'password':'bigMysql1234@'   
      }
}

conf_dm = {
      'url':'jdbc:mysql://localhost:3306/final_project_DM?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigDM',
      'password':'bigDM1234@'   
      }
}

In [4]:
# table 저장 / 반환 관련한 함수 구성
# config : db 연결 정보
# spark.read.jdbc() : db에서 명시된 테이블을 반환받는 모듈
def find_data(config, table_name) :
    return spark.read.jdbc(url= config['url'], table=table_name, properties=config['props'])

def save_data(config, df, table_name) :
    return df.write.jdbc(url= config['url'], table=table_name, mode='overwrite' , properties=config['props'])

## 1. 시나리오 1(rating prediction)
- reviews_table_category3 => ratings, review_contents 이용해서 진행 (NLP 모델링에 사용)

In [5]:
products_review_info = find_data(conf_dm, 'products_review_info')
products_table = find_data(conf_dm, 'products_table')
reviews_table = find_data(conf_dm, 'reviews_table')

In [6]:
reviews_table.createOrReplaceTempView('reviews_table')
products_table.createOrReplaceTempView('products_table')
products_review_info.createOrReplaceTempView('products_review_info')

In [7]:
scenario1 = spark.sql('''
    SELECT review_id, ratings, review_contents
    FROM reviews_table;
''')

In [8]:
scenario1.createOrReplaceTempView('scenario1')

In [9]:
spark.sql('''
    select count(*) from scenario1;
''').show()

+--------+
|count(1)|
+--------+
|  159244|
+--------+



In [10]:
save_data(conf_dm, scenario1, 'scenario1')

## 시나리오 2(recommendation)
- products_table_1, 2, 3 연결해서 3의 rating, review_cnt를이용해서 weight 부여 => product_name, product_url 

In [11]:
spark.sql('''
    select count(userid) from (select userid from reviews_table) where userid LIKE 'cust%';
''').show()

+-------------+
|count(userid)|
+-------------+
|          830|
+-------------+



### 로그인 한 사람 카테고리, 제품 추천


In [12]:
login_recommend = spark.sql('''
    SELECT tb1.review_id, tb1.userid, tb1.ratings, tb1.category1, tb1.category2, tb1.product_name, tb2.product_url
    FROM reviews_table tb1
    LEFT JOIN products_table tb2 ON tb1.product_id == tb2.product_id;
''')

In [13]:
login_recommend.createOrReplaceTempView('login_recommend')

In [14]:
spark.sql('''
    select count(*) from login_recommend;
''').show()

+--------+
|count(1)|
+--------+
|  159244|
+--------+



In [15]:
save_data(conf_dm, login_recommend, 'login_recommend')

### 로그인 안한 사람 제품, 카테고리 추천

In [16]:
no_login_recommend = spark.sql('''
    SELECT tb1.product_id, tb1.product_rating, tb1.review_count, tb2.category1, tb2.category2, tb2.product_name, tb2.product_url
    FROM products_review_info tb1
    JOIN products_table tb2 ON tb1.product_id == tb2.product_id
    WHERE tb2.category1 IN (
        SELECT category1
        FROM (
            SELECT category1, COUNT(*) as category_count
            FROM products_table
            GROUP BY category1
            HAVING category_count > 1
        )
    );
''')

In [17]:
no_login_recommend.createOrReplaceTempView('no_login_recommend')

In [18]:
spark.sql('''
    select count(*) from no_login_recommend;
''').show()

+--------+
|count(1)|
+--------+
|  108225|
+--------+



In [19]:
save_data(conf_dm, no_login_recommend, 'no_login_recommend')

## 시나리오3 (visualization)

In [20]:
wordcloud = spark.sql('''
    SELECT tb1.review_id, tb2.category1, tb2.category2, tb1.ratings, tb1.review_contents
    FROM reviews_table tb1
    JOIN products_table tb2 ON tb1.product_id == tb2.product_id;
''')

In [21]:
save_data(conf_dm, wordcloud, 'wordcloud')

In [22]:
products_review_visualization = spark.sql('''
    SELECT tb1.product_id, tb1.product_rating, tb1.review_count, tb2.product_name, tb2.category1, tb2.category2
    FROM products_review_info tb1
    JOIN products_table tb2 ON tb1.product_id == tb2.product_id;
''')

In [23]:
products_review_visualization.createOrReplaceTempView('products_review_visualization')

In [24]:
spark.sql('''
    select count(*) from products_review_visualization;
''').show()

+--------+
|count(1)|
+--------+
|  108225|
+--------+



In [25]:
save_data(conf_dm, products_review_visualization, 'products_review_visualization')